In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import cv2
import numpy as np
import os

In [3]:
# Set the path to the directory containing your images
image_folder = 'ChandrapurDaily'
video_name = 'output.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort()  # Sort the images if needed

# Read the first image to determine the frame size
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, 3, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [77]:

folder = 'ChandrapurDaily'

for img in os.listdir(folder):
    if img!="Chandrapur_20230319.png":
        continue
    image = cv2.imread(os.path.join(folder, img))

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # normalize the image
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    
    # Apply thresholding to isolate clouds
    # Adjust the threshold value as needed
    _, thresholded = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Optional: Filter contours by size
    min_area = 500  # minimum area of the cloud
    cloud_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Create a mask for the clouds
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, cloud_contours, -1, (255), thickness=cv2.FILLED)

    # Extract clouds from the original image (optional)
    clouds = cv2.bitwise_and(image, image, mask=mask)
    combined_frame = cv2.hconcat([cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), clouds])
    # Save or display the results
    # cv2.imwrite('clouds_extracted.jpg', combined_frame)
    cv2.imshow('Clouds', combined_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    break


In [79]:
# Read the two images
image1 = cv2.imread('ChandrapurDaily\Chandrapur_20230423.png')
image2 = cv2.imread('ChandrapurDaily\Chandrapur_20230319.png')
# image1 = cv2.convertScaleAbs(image1, alpha=2, beta=30)
# image2 = cv2.convertScaleAbs(image2, alpha=1.2, beta=0)

# Convert to grayscale
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)


# # add guassian blur 
# kernel_size = (5, 5)
# gray1 = cv2.GaussianBlur(gray1, kernel_size, cv2.BORDER_DEFAULT)
# gray2 = cv2.GaussianBlur(gray2, kernel_size, cv2.BORDER_DEFAULT)
# normalize with min max
gray1 = cv2.normalize(gray1, None, 0, 255, cv2.NORM_MINMAX)
gray2 = cv2.normalize(gray2, None, 0, 255, cv2.NORM_MINMAX)

hsv = np.zeros_like(image1)
hsv[...,1] = 255

# Initialize points to track
n_step = 5  # Number of points to track
points_to_track = np.array([[[x, y]] for y in range(0, gray1.shape[0], n_step)
                                         for x in range(0, gray1.shape[1], n_step)], dtype=np.float32)

# Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Calculate optical flow
points2, st, err = cv2.calcOpticalFlowPyrLK(gray1, gray2, points_to_track, None, **lk_params)
flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
# Convert to magnitude and angle
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
mag[mag < np.quantile(mag, 0.50)] = 0

# Set image hue according to the angle of optical flow
hsv[..., 0] = ang * (180 / np.pi / 2)
# hsv[..., 0] = 255
# Set image value according to the normalized magnitude of optical flow
hsv[..., 0] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
hsv[..., 1] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
# Convert HSV to RGB
rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
# cv2.imshow("Optical Flow_1", hsv)
# cv2.waitKey(0)

# Select good points
good_new = points2[st == 1]
good_old = points_to_track[st == 1]

# Draw tracks
output_image = cv2.cvtColor(gray1, cv2.COLOR_GRAY2BGR)
for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    a, b, c, d = int(a), int(b), int(c), int(d)  # Convert to integers
    cv2.line(output_image, (a, b), (c, d), (0, 255, 0), 1)
    cv2.circle(output_image, (a, b), 1, (0, 0, 255), -1)

combined_frame = cv2.vconcat([
                            # cv2.hconcat([image1, image2]),
                            cv2.hconcat([cv2.cvtColor(gray1, cv2.COLOR_GRAY2BGR), cv2.cvtColor(gray2, cv2.COLOR_GRAY2BGR)]),    
                            cv2.hconcat([hsv, output_image])])

cv2.imshow('Optical Flow', combined_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [21]:
np.quantile(ang, 0.15)

0.9034024477005005

In [6]:
cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).shape

(369, 369)

In [7]:
cv2.imshow("Optical Flow", cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX))


In [8]:
folder = 'ChandrapurDaily'
# Read the two images
base = cv2.imread('ChandrapurDaily\Chandrapur_20230423.png')
hsv = np.zeros_like(base)
hsv[...,1] = 255
# Convert to grayscale
base_grey = cv2.cvtColor(base, cv2.COLOR_BGR2GRAY)

height, width, layers = 369, 369, 3
video_name = "flow2.avi"
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, 3, (width, height))


folder = 'ChandrapurDaily'

for image in os.listdir(folder):
    image2 = cv2.imread(os.path.join(folder, image))
    # Convert to grayscale
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(base_grey, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Convert to magnitude and angle
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Set image hue according to the angle of optical flow
    hsv[..., 0] = ang * (180 / np.pi / 2)
    # Set image value according to the normalized magnitude of optical flow
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    # Convert HSV to RGB
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    # Display image
    cv2.imshow("Optical Flow", rgb)
    # cv2.waitKey(0)
    # Save image
    cv2.imwrite("optical_flow.png", rgb)
    video.write(rgb)    

cv2.destroyAllWindows()
video.release()

In [9]:
cap = cv2.VideoCapture("output.avi")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255


height, width, layers = 369, 369, 3
video_name = "flow.avi"
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(video_name, fourcc, 3, (width, height))


while(1):
    ret, frame2 = cap.read()
    # Check if frame is read correctly
    if not ret:
        break

    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    video.write(rgb)    
    prvs = next

cap.release()
cv2.destroyAllWindows()
video.release()


In [10]:
# not wortking
cap = cv2.VideoCapture('output.avi')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    # Check if frame is read correctly
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        a, b, c, d = int(a), int(b), int(c), int(d)  # Convert to integers
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

TypeError: 'NoneType' object is not subscriptable

: 

In [ ]:
# Open the two videos
cap1 = cv2.VideoCapture('output.avi')
cap2 = cv2.VideoCapture('flow2.avi')

# Check if videos opened successfully
if not cap1.isOpened() or not cap2.isOpened():
    print("Error opening video files")
    exit()

# Get properties from the videos
width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Determine the combined width and the height (maximum height of the two videos)
combined_width = width1 + width2
combined_height = max(height1, height2)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('combined_output.avi', fourcc, 2.0, (combined_width, combined_height))

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    # Check if both frames are read correctly
    if not ret1 or not ret2:
        break

    # Resize frames to have the same height
    if height1 != combined_height:
        frame1 = cv2.resize(frame1, (int(frame1.shape[1] * combined_height / frame1.shape[0]), combined_height))
    if height2 != combined_height:
        frame2 = cv2.resize(frame2, (int(frame2.shape[1] * combined_height / frame2.shape[0]), combined_height))

    # Concatenate frames horizontally
    combined_frame = cv2.hconcat([frame1, frame2])
    cv2.imshow('frame2',combined_frame)
    cv2.waitKey(0)
    

    # Write the combined frame
    out.write(combined_frame)

# Release everything if job is finished
cap1.release()
cap2.release()
out.release()
cv2.destroyAllWindows()
